<a href="https://colab.research.google.com/github/pi-aire/dataminig_sampling/blob/main/TP_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre





In [45]:
import numpy as np
import scipy.special as sps
import math
import matplotlib
import pandas as pd
import random

In [46]:
# Le dataframe
df = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del df[37]
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,21,23,25,27,29,31,34,36,38,40,42,44,47,48,50,52,54,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,51,52,54,56,58,60,62,64,66,68,70,72,74


## Question 1

In [181]:
def algoFrequences(df:pd.DataFrame,nb_pattern)-> tuple:
  #R:set = set()
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df[0].count())
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,len(df.iloc[i]))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selctionne un motif 
    pattern = np.array(df.iloc[row])
    random_v = random.randint(1, len(df.iloc[row]) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifes non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

On calcule le poids total de toutes les lignes dans un premier temps afin de diviser le poids des lignes dans le tableau.

In [ ]:
algoFrequences(df,10)

## Question 2

In [84]:
def algoArea(df:pd.DataFrame,nb_pattern)-> tuple:
  R = []
  IsInR = set()
  
  # set les probas
  w = np.zeros(df[0].count())
  totalW = 0
  for i in range(1, len(w)):
    w[i] = len(df.iloc[i]) * (2 ** (len(df.iloc[i]) - 1))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    # On cherche la ligne
    random_row = random.uniform(0,totalW)
    v, row = 0,0
    for i in range(len(w)):
      if v > random_row:
        row = i - 1
        break
      v += w[i]
      
    # On set les probabilité de k (taille du motif)
    # On veut que le sous ensemble avec une probabilté 
    # proportinelle à la taille de la datarecord (ligne)
    ks = np.zeros(len(df.iloc[row]))
    totalK = 0
    for i in range(len(df.iloc[row])):
        ks[i] = sps.binom(len(df.iloc[row]), i + 1)
        totalK += ks[i]
    random_kp = random.uniform(0, totalK)
    # On cherche la ligne
    k, v = 0,0
    for i in range(len(df.iloc[row])):
      if v > random_kp:
        k = i - 1
        break
      v += ks[i]
    
    pattern = np.array(df.iloc[row])
    for i in range(len(pattern)- k):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [86]:
algoArea(df,10)

[array([44, 46,  2, 64, 72, 56, 60, 54, 74, 25, 72, 33, 70, 34,  3, 62, 48,
         9, 25]),
 array([31, 69, 12, 20, 12, 56, 36, 50, 55, 29, 40, 42,  9, 25, 40, 62, 58,
        56]),
 array([55, 66, 51, 70, 66, 56, 44, 64, 38,  2, 23, 36, 58, 42, 56, 66, 66,
        23, 48, 72]),
 array([ 1, 36, 50, 60, 14, 62, 23, 22, 45, 29,  9, 17, 11, 38, 28, 58, 22,
         1, 25, 52, 15, 52]),
 array([ 3, 23, 31, 36, 66, 75, 52, 40, 21, 42, 14,  3, 40, 34, 48, 60, 64,
        48, 58,  1]),
 array([23, 42, 25, 23, 74, 62,  3, 72, 52,  2, 28,  9, 62,  5, 31, 36, 16,
        16]),
 array([ 9, 19, 19, 44, 42, 54, 44, 46, 46,  7, 24, 74,  5, 38,  7, 56, 16,
        50,  7,  2]),
 array([ 4, 52, 12,  4, 66, 24, 29, 21, 48, 25, 25,  9, 21, 36, 58, 38, 66,
        62, 27, 20, 31, 38, 15, 62]),
 array([68, 68, 52, 17, 44,  5, 12, 36, 29, 17,  7, 68, 12, 17,  3, 74, 31,
        34, 60, 28,  5, 31, 70, 68, 60]),
 array([71, 48, 10, 74, 24, 36, 19, 58, 71, 58, 74, 54, 46, 69, 32,  3, 14,
        14])]

##Question 3

In [183]:
def frequences(df, patterns):
  frequencesP = np.zeros(len(patterns),dtype=int)
  for i in range(df.count()[0]):
    indexs = np.zeros(len(patterns),dtype=int)
    for j in range(df.iloc[i].count()):
      for ip in range(len(patterns)):
        #print(df.iloc[i][j],patterns[ip][indexs[ip]])
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            frequencesP[ip] += 1
        #print(indexs)
  return frequencesP

In [184]:
patterns = algoFrequences(df,10)
print(patterns)

28
33
14
21
24
8
8
24
36
3
[array([ 1,  3,  5, 14, 20, 21, 25, 27, 29, 32, 34, 36, 38, 42, 47, 48, 51,
       52, 54, 56, 58, 60, 62, 65, 66, 68, 70, 74]), array([ 2,  3,  5,  7, 10, 14, 15, 17, 21, 23, 25, 28, 29, 31, 34, 36, 38,
       40, 42, 44, 46, 48, 50, 52, 55, 57, 58, 60, 62, 64, 69, 70, 75]), array([ 1, 11, 17, 23, 25, 31, 36, 42, 48, 50, 56, 60, 69, 70]), array([ 3,  5,  7, 11, 14, 16, 17, 23, 25, 31, 34, 36, 39, 40, 42, 48, 56,
       62, 64, 66, 70]), array([ 2,  5,  7, 17, 19, 22, 24, 27, 29, 33, 34, 36, 38, 42, 48, 50, 54,
       56, 58, 62, 65, 66, 73, 74]), array([19, 27, 39, 42, 44, 52, 54, 74]), array([ 7, 21, 25, 29, 54, 60, 64, 72]), array([ 1,  5,  7,  9, 12, 13, 16, 19, 24, 34, 36, 40, 42, 44, 48, 51, 54,
       56, 60, 62, 66, 68, 70, 72]), array([ 1,  3,  6,  7,  9, 11, 13, 15, 17, 20, 21, 24, 25, 27, 29, 31, 34,
       36, 39, 40, 43, 45, 48, 50, 52, 54, 57, 58, 60, 63, 65, 66, 69, 71,
       73, 74]), array([11, 56, 64])]


In [186]:
frequences(df,patterns)

array([   1,    1,   34,    2,    1,  200,  609,    3,    1, 1604])

## Question 4

In [ ]:
dataset_1 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
dataset_1.head()

In [202]:
dataset_2 = pd.read_fwf("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/kdd.txt", sep=" ",header=None,)
dataset_2 = dataset_2[0].str.split(' ', expand=True)
dataset_2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
0,80,209,217,291,345,495,551,553,557,805,949,982,1148,1257,1269,1339,1475,1481,1526,1624,1803,1983,1993,1998,2005,2022,2114,2167,2323,2345,2425,2516,2686,2709,2781,2862,2872,2911,3041,3077,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,9,143,198,316,337,362,411,418,421,596,734,785,833,918,1039,1207,1232,1307,1340,1473,1488,1543,1567,2035,2227,2253,2524,2781,2852,2872,2943,2992,3064,3079,3115,3120,3207,3255,3354,3386,...,6144,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,204,286,291,676,722,724,788,805,944,982,1154,1312,1339,1488,1582,1616,1746,1991,2174,2426,2686,2696,2724,2852,2863,2906,2910,2967,3041,3077,3109,3321,3323,3446,3764,3787,3835,3865,3962,3985,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,39,286,291,384,452,694,713,1012,1029,1312,1339,1460,1537,1823,1939,2068,2086,2114,2152,2325,2330,2377,2425,3045,3064,3077,3120,3285,3299,3332,3368,3441,3508,3672,3744,3751,3752,3828,3846,3973,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,44,179,676,717,807,982,1339,1811,1970,1988,2062,2071,2096,2280,2363,2425,2709,2777,3065,3077,3115,3163,3255,3272,3315,3403,3446,3479,3515,3574,3635,4200,4325,4372,4527,4538,4979,4980,5247,5258,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
dataset_3 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/MSNBC.txt", sep=" ",header=None,)
dataset_3 = dataset_3[0].str.split(' ', expand=True)
dataset_3.head()

In [ ]:
dataset_4 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/BIBLE.txt", sep=" ",header=None,)
dataset_4 = dataset_4[0].str.split(' ', expand=True)
dataset_4.head()

In [ ]:
da

## Question 5

### Etude emprique

## Question 6